# Critical EDF Statistics
In this Jupyter notebook, critical empirical distribution function (EDF) statistics for the investigated two-parameter probability distributions are computed. The statistics used are the Kolmogorov-Smirnov statistic for the Lilliefors test and the Anderson-Darling statistic.

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
import json
import numpy as np
from pathlib import Path
from cmcrameri.cm import *
import matplotlib.pyplot as plt
from pdtoolbox.gof import LillieforsTable, AndersonDarlingTable
from pdtoolbox import ExtremeValue2Distribution, FrechetDistribution, \
                      GammaDistribution, InverseGammaDistribution, \
                      LogLogisticDistribution, LogNormalDistribution, \
                      NakagamiDistribution, NormalDistribution, \
                      ShiftedGompertzDistribution, WeibullDistribution

We work consistently at $\alpha=0.05$ in this whole notebook.

In [ ]:
ALPHA = 0.05

Number of Monte-Carlo samples per critical table entry:

In [ ]:
N_MC = 50000

In [ ]:
N = np.arange(10, 120)

In [ ]:
def compute_critical_tables(path, dist, *args, alpha=ALPHA, N=N, Nmc=N_MC, mle_kwargs={}):
    path = Path(path)
    if path.is_file():
        with open(path, 'r') as f:
            LA = json.load(f)
        return LillieforsTable.from_json(LA[0]), AndersonDarlingTable.from_json(LA[1])
    else:
        L,A = dist.critical_tables(ALPHA, N, *args, Nmc=N_MC, verbose=True,
                                   **mle_kwargs)
        with open(path, 'w') as f:
            json.dump((L.to_json(), A.to_json()), f)
        return L, A

## Extreme Value Distribution
The following line can be used to remove the previous results for the purpose of recalculation.
Convert it to a code cell and run.

Otherwise, the table will be loaded from the JSON file.

In [ ]:
LE, ADE = compute_critical_tables("intermediate/A1-Critical-Extreme2.json",
                                  ExtremeValue2Distribution)

## Fréchet Distribution

In [ ]:
LF, ADF = compute_critical_tables("intermediate/A1-Critical-Frechet.json",
                                  FrechetDistribution)

## Gamma Distribution
The gamma distribution has a scale and a shape parameter. While the distribution of the test statistic is invariant to the scale parameter (see Lilliefors (1957)), the shape parameter changes the critical values of the test statistic. Hence, we have to compute critical values of the test statistic for the whole range of the shape parameter that we might later encounter:

In [ ]:
K = np.concatenate([np.geomspace(1.0, 50.0, 10, endpoint=False),
                    np.geomspace(50.0, 1e8, 10)])

In [ ]:
LG, ADG = compute_critical_tables("intermediate/A1-Critical-Gamma.json",
                                  GammaDistribution, K, mle_kwargs={"kmin" : 1.0})

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.pcolormesh(N, K, LG._table.T, cmap=batlow)
ax.contour(N, K, LG._table.T, colors='w', linewidths=0.8)
ax.set_yscale('log')
ax.scatter(*np.meshgrid(N,K), marker='.', facecolor='k', edgecolor='none', s=4)
LG._table

## Inverse Gamma Distribution

In [ ]:
IG_ALPHA = np.concatenate([np.geomspace(1e-2, 50.0, 10, endpoint=False),
                           np.geomspace(50.0, 1e8, 10)])

In [ ]:
LIG, ADIG = compute_critical_tables("intermediate/A1-Critical-Inverse-Gamma.json",
                                    InverseGammaDistribution, IG_ALPHA)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.pcolormesh(N, IG_ALPHA, LIG._table.T, cmap=batlow)
ax.contour(N, IG_ALPHA, LIG._table.T, colors='w', linewidths=0.8)
ax.set_yscale('log')
ax.scatter(*np.meshgrid(N, IG_ALPHA), marker='.', facecolor='k', edgecolor='none', s=4)

## Log-Logistic Distribution

In [ ]:
LLL, ADLL = compute_critical_tables("intermediate/A1-Critical-Log-Logistic.json",
                                    LogLogisticDistribution)

## LogNormalDistribution

In [ ]:
LLN, ADLN = compute_critical_tables("intermediate/A1-Critical-Log-Normal.json",
                                    LogNormalDistribution)

## Nakagami Distribution

In [ ]:
m = np.concatenate([np.geomspace(0.5, 50.0, 10, endpoint=False),
                    np.geomspace(50.0, 1e8, 10)])

In [ ]:
LNAK, ADNAK = compute_critical_tables("intermediate/A1-Critical-Nakagami.json",
                                      NakagamiDistribution, m)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.pcolormesh(N, m, LNAK._table.T, cmap=batlow)
ax.contour(N, m, LNAK._table.T, colors='w', linewidths=0.8)
ax.set_yscale('log')
ax.scatter(*np.meshgrid(N,m), marker='.', facecolor='k', edgecolor='none', s=4)

## Normal Distribution

In [ ]:
LN, ADN = compute_critical_tables("intermediate/A1-Critical-Normal.json",
                                  NormalDistribution)

## Shifted Gompertz Distribution

In [ ]:
ETA = np.concatenate(([0], np.geomspace(1e-2, 1e8, 30), [1e10, 1e20, 1e40]))

In [ ]:
LSG, ADSG = compute_critical_tables("intermediate/A1-Critical-Shifted-Gompertz.json",
                                    ShiftedGompertzDistribution, ETA,
                                    mle_kwargs={"eta_max" : 1e40})

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.pcolormesh(N, ETA, LSG._table.T, cmap=batlow)
ax.contour(N, ETA, LSG._table.T, colors='w', linewidths=0.8)
ax.set_yscale('log')
ax.scatter(*np.meshgrid(N,ETA), marker='.', facecolor='k', edgecolor='none', s=4)

## Weibull Distribution

In [ ]:
LW, ADW = compute_critical_tables("intermediate/A1-Critical-Weibull.json",
                                  WeibullDistribution)

### License
```
A notebook to compute critical tables for goodness-of-fit tests
using the pdtoolbox module.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```